In [1]:
%%capture
!sudo curl -L https://github.com/dolthub/dolt/releases/latest/download/install.sh | sudo bash
!pip install pymysql
!dolt clone onefact/paylesshealth
  

In [2]:
!cd paylesshealth/ && dolt sql -q "select * from hospitals where cdm_url is not null;" -r csv > hospitals.csv

In [48]:
import pandas as pd
# df = pd.read_csv('paylesshealth/hospitals.csv')
df = pd.read_csv('non-null.csv')
# df = df.head()

In [31]:
import aiohttp
import asyncio
import re
import cgi
from pathlib import Path
from urllib.parse import urlparse

Task exception was never retrieved
future: <Task finished name='Task-28894' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 560, in _request
    await resp.start(conn)
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-p

Task exception was never retrieved
future: <Task finished name='Task-29862' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-31172' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-28845' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-26953' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

...

Task exception was never retrieved
future: <Task finished name='Task-28923' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-28752' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-29099' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-27786' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

Task exception was never retrieved
future: <Task finished name='Task-29539' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py:75> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_27608\3299729886.py", line 22, in checker
    async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "c:\users\adria\miniconda3\envs\scraper\lib\site-packages\aiohttp\client.py", line 536, in _request
    conn = await self._connector.connect(
  File "c:\users\adria\miniconda3\envs\scra

In [6]:
def is_url(url):
    try:
        a = urlparse(url)
        return all([a.scheme, a.netloc])
    except ValueError:
        return False

In [53]:
async def checker(session, url):
    if pd.isna(url):
        return pd.NA
    
    if "/../" in url or "|" in url: # aiohttp is kinda not smart
        return url

    
    headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Host": urlparse(url).netloc,
    "Referer": urlparse(url).netloc
    }
    try:
        async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
            if r.status == 200:
                return url
            elif r.status == 404:
                return pd.NA
            elif r.status == 301 or r.status == 302:
                new_loc = r.headers.get('location')
                if is_url(new_loc):
                    return new_loc
                else:
                    return pd.NA
            elif r.status != 200 and r.status != 404:
#                 print(f"\nUnkknown code: {r.status} {url}")
                return url

    except aiohttp.ClientResponseError as e:
#         print("ERROR", e, url)
        return url

    except asyncio.TimeoutError:
        return url

    except aiohttp.ClientConnectionError:
        return url    
    
    except KeyboardInterrupt:
        import os; os.exit()
    except Exception:
        return url

In [38]:
from tqdm.asyncio import tqdm

async def main():
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        cols = ["cdm_url", "homepage", "cdm_indirect_url"]
        for col in cols:
            df[col] = await tqdm.gather(*(checker(session, url) for url in df[col]), leave=True, position=0)
        return df

In [54]:
t = await main()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4892/4892 [02:30<00:00, 32.52it/s]


In [56]:
t.to_csv("checked_urls.csv", index=False)